In [1]:
!nvidia-smi

Sun Oct  5 01:23:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip -q install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.6 MB/s eta 0:00:00


In [6]:
import yt_dlp
from pathlib import Path
import pandas as pd
import json, os
from pathlib import Path
from IPython.display import Video
import ast
import cv2
from tqdm import tqdm
import subprocess
from ultralytics import YOLO
import numpy as np



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
CSV_PATH = "/content/gait_index.csv"   # ajuste se precisar
df = pd.read_csv(CSV_PATH, sep=";")

# (opcional) padronizar labels e filtrar visão
# df['gait_pat'] = df['gait_pat'].str.strip().str.lower()


df.drop(columns=['vid_info', 'bbox'], inplace=True)
df = df[df['cam_view'].str.contains('side|lateral', case=False, na=False)]
df['min_frame'] = df.groupby('id')['frame_num'].transform('min')
df['max_frame'] = df.groupby('id')['frame_num'].transform('max')

df.drop_duplicates(subset=['id'], inplace=True)
df.set_index('id', inplace=True)
df = df.head(3)
df

,seq,frame_num,cam_view,gait_event,dataset,gait_pat,url,min_frame,max_frame
id,,,,,,,,,
B5hrxKe2nP8,cljan9b4p00043n6ligceanyp,1757,right side,NaN,Abnormal Gait,parkinsons,https://www.youtube.com/watch?v=B5hrxKe2nP8,1757.0,2746.0
TgkxrrhnvlM,cljao8kyf000d3n6l0x9kgmav,1,right side,NaN,Abnormal Gait,abnormal,https://www.youtube.com/watch?v=TgkxrrhnvlM,1.0,1342.0
0gAHlm79Bjo,cljapao5600503n6lhrbr4zii,1,left side,NaN,Abnormal Gait,abnormal,https://www.youtube.com/watch?v=0gAHlm79Bjo,1.0,659.0


In [9]:
VID_DIR = Path("/content/videos-out"); VID_DIR.mkdir(exist_ok=True, parents=True)

def download_video(id, row):
    url = row['url'] if pd.notna(row.get('url')) else f"https://www.youtube.com/watch?v={id}"
    out = VID_DIR / f"{id}.mp4"
    ydl_opts = {
        "outtmpl": str(out),
        "format": "bestvideo[ext=mp4]",
        "merge_output_format": "mp4",
        "quiet": True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

    fps = info.get("fps")
    if fps is None:
        fps = 30.0

    return str(out), float(fps)

def download_all(df):
  for id, row in tqdm(df.iterrows(), total=len(df)):
    path, fps = download_video(id, row)
    df.at[id, "video_path"] = path
    df.at[id, "fps"] = fps
  return df

df = download_all(df)
df.head()


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:09<00:18,  9.13s/it]

 67%|██████▋   | 2/3 [00:20<00:10, 10.58s/it]

100%|██████████| 3/3 [00:32<00:00, 10.90s/it]


,seq,frame_num,cam_view,gait_event,dataset,gait_pat,url,min_frame,max_frame,video_path,fps
id,,,,,,,,,,,
B5hrxKe2nP8,cljan9b4p00043n6ligceanyp,1757,right side,NaN,Abnormal Gait,parkinsons,https://www.youtube.com/watch?v=B5hrxKe2nP8,1757.0,2746.0,/content/videos-out/B5hrxKe2nP8.mp4,30.0
TgkxrrhnvlM,cljao8kyf000d3n6l0x9kgmav,1,right side,NaN,Abnormal Gait,abnormal,https://www.youtube.com/watch?v=TgkxrrhnvlM,1.0,1342.0,/content/videos-out/TgkxrrhnvlM.mp4,60.0
0gAHlm79Bjo,cljapao5600503n6lhrbr4zii,1,left side,NaN,Abnormal Gait,abnormal,https://www.youtube.com/watch?v=0gAHlm79Bjo,1.0,659.0,/content/videos-out/0gAHlm79Bjo.mp4,30.0


In [10]:
Video(df.iloc[0]['video_path'], embed=True, width=500, height=500)

Output hidden; open in https://colab.research.google.com to view.

In [11]:

df.head()

,seq,frame_num,cam_view,gait_event,dataset,gait_pat,url,min_frame,max_frame,video_path,fps,start_time,end_time
id,,,,,,,,,,,,,
B5hrxKe2nP8,cljan9b4p00043n6ligceanyp,1757,right side,NaN,Abnormal Gait,parkinsons,https://www.youtube.com/watch?v=B5hrxKe2nP8,1757.0,2746.0,/content/videos-out/B5hrxKe2nP8.mp4,30.0,58.567,91.533
TgkxrrhnvlM,cljao8kyf000d3n6l0x9kgmav,1,right side,NaN,Abnormal Gait,abnormal,https://www.youtube.com/watch?v=TgkxrrhnvlM,1.0,1342.0,/content/videos-out/TgkxrrhnvlM.mp4,60.0,0.017,22.367
0gAHlm79Bjo,cljapao5600503n6lhrbr4zii,1,left side,NaN,Abnormal Gait,abnormal,https://www.youtube.com/watch?v=0gAHlm79Bjo,1.0,659.0,/content/videos-out/0gAHlm79Bjo.mp4,30.0,0.033,21.967


In [12]:
OUT_DIR = Path("/content/videos-crops"); OUT_DIR.mkdir(exist_ok=True, parents=True)

for index, row in df.iterrows():
  input_path = row['video_path']
  output_path = OUT_DIR / f"{index}_crop.mp4"
  start_time = row['start_time']
  end_time = row['end_time']
  fps = row['fps']

  print(f"Cortando {input_path} do tempo {start_time} ao {end_time}")

  # Monta o comando ffmpeg
  cmd = [
      "ffmpeg",
      "-r", str(fps),
      "-y",  # sobrescreve se já existir
      "-i", input_path,
      "-ss", str(start_time),
      "-to", str(end_time),
      "-c:v", "h264_nvenc",
      output_path
  ]

  # Executa o comando
  subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  df.at[index, 'crop_path'] = output_path
  print(f"Gerado: {output_path}")

Cortando /content/videos-out/B5hrxKe2nP8.mp4 do tempo 58.567 ao 91.533
Gerado: /content/videos-crops/B5hrxKe2nP8_crop.mp4
Cortando /content/videos-out/TgkxrrhnvlM.mp4 do tempo 0.017 ao 22.367
Gerado: /content/videos-crops/TgkxrrhnvlM_crop.mp4
Cortando /content/videos-out/0gAHlm79Bjo.mp4 do tempo 0.033 ao 21.967
Gerado: /content/videos-crops/0gAHlm79Bjo_crop.mp4


In [13]:
Video(df.iloc[0]["crop_path"], embed=True, width=500, height=500)

Output hidden; open in https://colab.research.google.com to view.

In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00


In [14]:
from scipy.signal import savgol_filter

def clean_kps_seq(kps_seq):
    """
    Limpa e normaliza uma sequência de keypoints [T,17,3]
    Retorna uma nova sequência com (x,y) centralizados e normalizados.
    """
    kps = kps_seq.copy()

    # 1. Substituir NaNs por interpolação linear
    for j in range(kps.shape[1]*2):  # x e y de cada ponto
        t_series = kps[:, j//2, j%2]
        nans = np.isnan(t_series)
        if nans.any():
            idx = np.arange(len(t_series))
            if (~nans).sum() > 1:
                t_series[nans] = np.interp(idx[nans], idx[~nans], t_series[~nans])
            else:
                t_series[nans] = 0
        kps[:, j//2, j%2] = t_series

    # 2. Zerar keypoints com confiança muito baixa
    conf_mask = kps[:,:,2] < 0.2
    kps[conf_mask, :2] = np.nan

    # 3. Centralizar no quadril (média dos keypoints 11 e 12 - padrão COCO)
    left_hip, right_hip = 11, 12
    center = np.nanmean(kps[:, [left_hip, right_hip], :2], axis=1)
    kps[:, :, :2] -= center[:, np.newaxis, :]

    # 4. Normalizar pela distância entre ombros (escala corporal)
    left_sh, right_sh = 5, 6
    shoulder_dist = np.linalg.norm(kps[:, left_sh, :2] - kps[:, right_sh, :2], axis=1)
    scale = np.nanmedian(shoulder_dist)
    kps[:, :, :2] /= (scale + 1e-6)

    # 5. Suavizar as trajetórias
    for j in range(kps.shape[1]*2):
        seq = kps[:, j//2, j%2]
        if len(seq) >= 9:
            kps[:, j//2, j%2] = savgol_filter(seq, window_length=9, polyorder=2)

    return kps

In [15]:
model = YOLO("yolo11n-pose.pt")
POSES_DIR = Path("/content/poses"); POSES_DIR.mkdir(exist_ok=True, parents=True)
VIDEOS_POSES = Path("/content/videos-poses"); VIDEOS_POSES.mkdir(exist_ok=True, parents=True)


for id, row in tqdm(df.iterrows(), total=len(df)):
  kps_seq = []  # [T,17,3]
  source = row['crop_path']
  print(f"Processando {source}")
  save = False
  if id == df.index[0]:
    save = True

  pred = model.predict(
      source=source,
      stream=True,
      conf=0.25,
      save=save,
      device=0,
      project=VIDEOS_POSES
  )

  for r in pred:
    if r.keypoints is None or len(r.keypoints)==0:
        kps_seq.append(np.full((17,3), np.nan, dtype=np.float32))
        continue

    # escolher a pessoa principal (maior bbox)
    areas = (r.boxes.xywh[:,2] * r.boxes.xywh[:,3]).cpu().numpy()
    i = int(areas.argmax())

    kps_seq.append(r.keypoints.data[i].cpu().numpy())  # (17,3)

  kps_seq = np.array(kps_seq, dtype=np.float32)
  kps_seq_clean = clean_kps_seq(kps_seq)
  np.save(POSES_DIR / f"{id}_poses.npy", kps_seq_clean)

  0%|          | 0/3 [00:00<?, ?it/s]

Processando /content/videos-crops/B5hrxKe2nP8_crop.mp4

video 1/1 (frame 1/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 55.1ms
video 1/1 (frame 2/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.4ms
video 1/1 (frame 3/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.5ms
video 1/1 (frame 4/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.3ms
video 1/1 (frame 5/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.6ms
video 1/1 (frame 6/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.7ms
video 1/1 (frame 7/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.2ms
video 1/1 (frame 8/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 8.8ms
video 1/1 (frame 9/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 9.2ms
video 1/1 (frame 10/989) /content/videos-crops/B5hrxKe2nP8_crop.mp4: 384x640 1 person, 12.8ms
video 1/1 (frame 11/9

 33%|███▎      | 1/3 [00:44<01:29, 44.73s/it]

Processando /content/videos-crops/TgkxrrhnvlM_crop.mp4

video 1/1 (frame 1/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 8.8ms
video 1/1 (frame 2/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 9.9ms
video 1/1 (frame 3/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 9.2ms
video 1/1 (frame 4/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 11.6ms
video 1/1 (frame 5/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 9.4ms
video 1/1 (frame 6/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 11.5ms
video 1/1 (frame 7/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 10.3ms
video 1/1 (frame 8/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 9.9ms
video 1/1 (frame 9/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 10.0ms
video 1/1 (frame 10/1341) /content/videos-crops/TgkxrrhnvlM_crop.mp4: 384x640 1 person, 9.9ms
video 1/1

 67%|██████▋   | 2/3 [01:14<00:35, 35.91s/it]

Processando /content/videos-crops/0gAHlm79Bjo_crop.mp4

video 1/1 (frame 1/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 9.3ms
video 1/1 (frame 2/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 11.8ms
video 1/1 (frame 3/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 9.9ms
video 1/1 (frame 4/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 9.9ms
video 1/1 (frame 5/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 12.4ms
video 1/1 (frame 6/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 11.3ms
video 1/1 (frame 7/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 10.5ms
video 1/1 (frame 8/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 12.3ms
video 1/1 (frame 9/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 10.1ms
video 1/1 (frame 10/658) /content/videos-crops/0gAHlm79Bjo_crop.mp4: 384x640 1 person, 11.6ms
video 1/1 (frame

100%|██████████| 3/3 [01:31<00:00, 30.46s/it]


In [16]:
input_pose = VIDEOS_POSES / f'predict/{df.iloc[0].name}_crop.avi'
output_pose = VIDEOS_POSES / f'predict/{df.iloc[0].name}_pose.mp4'
cmd = [
  "ffmpeg",
  "-i", input_pose,
  "-vcodec", "libx264",
  output_pose
]
subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

CompletedProcess(args=['ffmpeg', '-i', PosixPath('/content/videos-poses/predict/B5hrxKe2nP8_crop.avi'), '-vcodec', 'libx264', PosixPath('/content/videos-poses/predict/B5hrxKe2nP8_pose.mp4')], returncode=0, stdout=b'', stderr=b"ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers\n  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitm

In [21]:
Video(output_pose, embed=True, width=400, height=400)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
lodaded = np.load(POSES_DIR / f"{df.index[0]}_poses.npy")
lodaded.shape

(989, 17, 3)